In [10]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py


In [2]:
t = load_dir_reduced_to_time("/data/new_trap/20200320/Bead1/Shaking/Shaking378/","Shaking3",max_files=1)

100%|██████████| 1/1 [00:08<00:00,  8.39s/it]


In [19]:
month = 11

In [21]:
print("I am a month %s" %month.zfill(2))

AttributeError: 'int' object has no attribute 'zfill'